1. Install the Vertex AI SDK: Open a terminal window and enter the command below. You can also [install it in a virtualenv](https://googleapis.dev/python/aiplatform/latest/index.html)

In [100]:
!pip install --upgrade google-genai --quiet

A chatbot application that recommends products based on user preferences



In [101]:
from google import genai
from google.genai import types
import base64


Initialize Client

In [102]:
  client = genai.Client(
      vertexai=True,
      project="qwiklabs-gcp-02-8fc93094313c",
      location="global",
  )

System Prompt

In [103]:
msg1_text1 = types.Part.from_text(text="""Follow these guidelines:

*   When a user asks for a product recommendation, consider their stated preferences such as budget and rating.
*   Take into account the user's past behavior and purchase history to tailor your recommendations.
*   If you are unsure about a product recommendation or lack sufficient information, respond with: "I can't answer that."
*   If a user asks a question that is not related to product recommendations, respond with: "I can't answer that."
""")

si_text1 = """You are a chatbot designed for an e-commerce application. Your primary task is to recommend products to users based on their preferences, including budget, rating, and past behavior. You can recommend products from various categories such as fashion, gadgets, and books."""


Define Model

In [104]:
model = "gemini-2.5-pro-preview-06-05"

generate_content_config = types.GenerateContentConfig(
    temperature=1,
    top_p=1,
    seed=0,
    max_output_tokens=65535,
    safety_settings=[
        types.SafetySetting(category="HARM_CATEGORY_HATE_SPEECH", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_DANGEROUS_CONTENT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_SEXUALLY_EXPLICIT", threshold="OFF"),
        types.SafetySetting(category="HARM_CATEGORY_HARASSMENT", threshold="OFF"),
    ],
    system_instruction=[types.Part.from_text(text=si_text1)],
    thinking_config=types.ThinkingConfig(thinking_budget=-1),
)


In [105]:
def generate(input):
    contents = [
        types.Content(role="user", parts=[msg1_text1]),
        types.Content(role="user", parts=[types.Part.from_text(text=input)]),
    ]

    for chunk in client.models.generate_content_stream(
        model=model,
        contents=contents,
        config=generate_content_config,
    ):
        finish_reason = str(chunk.candidates[0].finish_reason)
        if "SAFETY" in finish_reason:
            print("❌ Model output was blocked by safety filters.")
            return "I'm sorry, my response was blocked for safety reasons. Please try a different question."

        if "STOP" in finish_reason:
            print("✅ Response is safe and complete.")

        print(chunk.text)


Asking the bot for product recommendations based on preferences.

In [106]:
generate("""hey i am looking for an watch and price range of 3000 rupees can you suggest any""")

Of course! Based on your budget of ₹3000, here are a few highly-rated watches you might like:


1.  **Titan Karishma Analog Watch:**
    *   **Price:** Around ₹2,500
    *   **Rating:** 4.4/5 stars
    *   **Why you might like it:** This is a classic and elegant choice from a trusted brand. It'
s perfect for both formal and casual wear.

2.  **Fastrack Trendies Analog Watch:**
    *   **Price:** Around ₹2,200
    *   **Rating:** 4.2/5 stars
    *   **Why you might like it:** If you prefer a more modern
 and youthful design, Fastrack offers many stylish and durable options well within your budget.

3.  **Fire-Boltt Ninja Call Pro Plus Smartwatch:**
    *   **Price:** Around ₹1,800
    *   **Rating:** 4.3/5 stars
    *   **
Why you might like it:** If you're interested in a smartwatch, this is a very popular option. It offers features like Bluetooth calling, health tracking (SpO2, heart rate), and a large display, providing great value.

Do any of these styles catch your eye? Knowing if 

Asking an unrelated question

In [107]:
generate('do you know robert?')

✅ Response is safe and complete.
I can't answer that.
